In [1]:
import pandas as pd
import numpy as np

In [2]:
# 上证指数已实现波动率计算
df_fminSZZS = pd.read_csv(r'C:\Users\jxjsj\Desktop\JupyterHome\Data\SZZS_five_min080101-181201.csv',index_col=0,header=0)

df_fminSZZS['time'] = list(df_fminSZZS.index)

df_fminSZZS['date'] = list(df_fminSZZS['time'].apply(lambda x: str(x)[:10]))

df_fminSZZS['close-1'] = df_fminSZZS['close'].shift(1)

df_fminSZZS_lnR_temp = pd.DataFrame(np.log(df_fminSZZS['close'])-np.log(df_fminSZZS['close-1']),index=df_fminSZZS.index,columns=['lnR'])

In [4]:
df_fminSZZS_lnR = pd.concat([df_fminSZZS_lnR_temp,pd.DataFrame(list(df_fminSZZS['date']),columns=['date'],index=df_fminSZZS.index)],axis=1).dropna(axis=0,how='any')

df_fminSZZS_lnR['lnR^2'] = df_fminSZZS_lnR['lnR'].apply(lambda x: x*x)

In [151]:
# t天预测t+1天
save_array = np.zeros(shape=(int((len(df_fminSZZS_lnR)-47)/48-21),48))
column_n = 0
row_n = -1
date_record = 0
for index in df_fminSZZS_lnR.index[47:-21*48]:
    if date_record == str(index)[:10]:
        save_array[row_n,column_n] = df_fminSZZS_lnR['lnR'][index]
        column_n += 1
    else:
        date_record = str(index)[:10]
        row_n += 1
        column_n = 0
        save_array[row_n,column_n] = df_fminSZZS_lnR['lnR'][index]
        column_n += 1

In [157]:
# t天上半天预测t天全天
save_array = np.zeros(shape=(int((len(df_fminSZZS_lnR)-47)/48-21),24))
column_n = 0
row_n = -1
date_record = 0
for index in df_fminSZZS_lnR.index[47:-21*48]:
    try:
        if date_record == str(index)[:10]:
            save_array[row_n,column_n] = df_fminSZZS_lnR['lnR'][index]
            column_n += 1
        else:
            date_record = str(index)[:10]
            row_n += 1
            column_n = 0
            save_array[row_n,column_n] = df_fminSZZS_lnR['lnR'][index]
            column_n += 1
    except:
        column_n += 1

In [161]:
# def positive_negative(x):
#     if x>=0.009364169774957184:
#         return 2
#     if x>=0.002597620432019096 and x<0.009364169774957184:
#         return 1
#     if x<-0.009043426461761683:
#         return -2
#     if x>=-0.009043426461761683 and x<-0.001910275079918831:
#         return -1
#     else:
#         return 0

def positive_negative(x):
    if x>=0:
        return 1
    else:
        return 0

data = pd.read_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/SZZS_lnr_rv_w_m_ntd_080101-181101.csv',index_col=0,header=0)
lnR = pd.DataFrame(data['lnR'])
lnR['lnR'] = lnR['lnR'].apply(lambda x: positive_negative(x))

In [147]:
save_array = save_array[:2634]
lnR = lnR[1:2635]

In [133]:
np.percentile(lnR, 100)

0.09034457846970413

In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score

In [162]:
x_train, x_test, y_train, y_test = train_test_split(save_array, lnR, random_state=1, train_size=0.8)

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [163]:
# 运用原非平衡样本-训练
realvolEvaluation = MLPClassifier(activation='tanh', 
                    solver='lbfgs',
                    alpha=1e-5,
                    hidden_layer_sizes=(len(x_train[0])*5+1,),
                    random_state=1, 
                   )
realvolEvaluation.fit(x_train,y_train)

# 运用原非平衡样本-评估
y_train_predict = realvolEvaluation.predict(x_train)
y_test_predict = realvolEvaluation.predict(x_test)

print('trainAccracy:',realvolEvaluation.score(x_train,y_train))
print(classification_report(y_train,y_train_predict)) #真实数据在前 训练结果在后！
print('testAccracy:',realvolEvaluation.score(x_test,y_test))
print(classification_report(y_test,y_test_predict))

D:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


trainAccracy: 0.7666034155597723
              precision    recall  f1-score   support

           0       0.77      0.73      0.75      1004
           1       0.76      0.80      0.78      1104

   micro avg       0.77      0.77      0.77      2108
   macro avg       0.77      0.76      0.77      2108
weighted avg       0.77      0.77      0.77      2108

testAccracy: 0.7476280834914611
              precision    recall  f1-score   support

           0       0.76      0.67      0.71       247
           1       0.74      0.82      0.77       280

   micro avg       0.75      0.75      0.75       527
   macro avg       0.75      0.74      0.74       527
weighted avg       0.75      0.75      0.75       527



In [164]:
from sklearn import svm
realvolEvaluation = svm.SVC(C=10, kernel='linear', decision_function_shape='ovr',class_weight = "balanced")

realvolEvaluation.fit(x_train,y_train)
# 运用原非平衡样本-评估
y_train_predict = realvolEvaluation.predict(x_train)
y_test_predict = realvolEvaluation.predict(x_test)

print('trainAccracy:',realvolEvaluation.score(x_train,y_train))
print(classification_report(y_train,y_train_predict)) #真实数据在前 训练结果在后！
print('testAccracy:',realvolEvaluation.score(x_test,y_test))
print(classification_report(y_test,y_test_predict))

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


trainAccracy: 0.6133776091081594
              precision    recall  f1-score   support

           0       0.80      0.25      0.38      1004
           1       0.58      0.94      0.72      1104

   micro avg       0.61      0.61      0.61      2108
   macro avg       0.69      0.60      0.55      2108
weighted avg       0.68      0.61      0.56      2108

testAccracy: 0.6053130929791272
              precision    recall  f1-score   support

           0       0.78      0.22      0.34       247
           1       0.58      0.95      0.72       280

   micro avg       0.61      0.61      0.61       527
   macro avg       0.68      0.58      0.53       527
weighted avg       0.67      0.61      0.54       527



In [165]:
from sklearn.naive_bayes import GaussianNB
realvolEvaluation = GaussianNB()

realvolEvaluation.fit(x_train,y_train)
# 运用原非平衡样本-评估
y_train_predict = realvolEvaluation.predict(x_train)
y_test_predict = realvolEvaluation.predict(x_test)

print('trainAccracy:',realvolEvaluation.score(x_train,y_train))
print(classification_report(y_train,y_train_predict)) #真实数据在前 训练结果在后！
print('testAccracy:',realvolEvaluation.score(x_test,y_test))
print(classification_report(y_test,y_test_predict))

trainAccracy: 0.6475332068311196
              precision    recall  f1-score   support

           0       0.73      0.41      0.53      1004
           1       0.62      0.86      0.72      1104

   micro avg       0.65      0.65      0.65      2108
   macro avg       0.67      0.64      0.62      2108
weighted avg       0.67      0.65      0.63      2108

testAccracy: 0.6451612903225806
              precision    recall  f1-score   support

           0       0.73      0.38      0.50       247
           1       0.62      0.88      0.72       280

   micro avg       0.65      0.65      0.65       527
   macro avg       0.68      0.63      0.61       527
weighted avg       0.67      0.65      0.62       527



D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [173]:
from sklearn import tree
realvolEvaluation = tree.DecisionTreeClassifier(criterion='gini', 
                                                max_depth=10, 
                                                min_samples_split=10, 
                                                min_samples_leaf =1, 
                                                min_weight_fraction_leaf=0.0, 
                                                max_features=None, 
                                                random_state=1, 
                                                max_leaf_nodes=None, 
                                                class_weight='balanced', 
                                                presort=False)

realvolEvaluation.fit(x_train,y_train)
# 运用原非平衡样本-评估
y_train_predict = realvolEvaluation.predict(x_train)
y_test_predict = realvolEvaluation.predict(x_test)

print('trainAccracy:',realvolEvaluation.score(x_train,y_train))
print(classification_report(y_train,y_train_predict)) #真实数据在前 训练结果在后！
print('testAccracy:',realvolEvaluation.score(x_test,y_test))
print(classification_report(y_test,y_test_predict))

trainAccracy: 0.9169829222011385
              precision    recall  f1-score   support

           0       0.91      0.92      0.91      1004
           1       0.92      0.92      0.92      1104

   micro avg       0.92      0.92      0.92      2108
   macro avg       0.92      0.92      0.92      2108
weighted avg       0.92      0.92      0.92      2108

testAccracy: 0.6546489563567363
              precision    recall  f1-score   support

           0       0.63      0.63      0.63       247
           1       0.67      0.68      0.68       280

   micro avg       0.65      0.65      0.65       527
   macro avg       0.65      0.65      0.65       527
weighted avg       0.65      0.65      0.65       527

